In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
# Essential DS libraries
import numpy as np
import pandas as pd
from sklearn.metrics import median_absolute_error, accuracy_score,roc_auc_score
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau

# LightAutoML presets, task and report generation
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.


/home/user/conda/lib/python3.9/site-packages/lightautoml/transformers/text.py:22: UserWarning: 'gensim' - package isn't installed
  warnings.warn("'gensim' - package isn't installed")


In [3]:
# utils

def map_class(x, task, reader):
    if task.name == 'multiclass':
        return reader[x]
    else:
        return x

mapped = np.vectorize(map_class)

def score(task, y_true, y_pred):
    if task.name == 'binary':
        return roc_auc_score(y_true, y_pred)
    elif task.name == 'multiclass':
        return accuracy_score(y_true, np.argmax(y_pred, 1))
    elif task.name == 'reg' or task.name == 'multi:reg':
        return median_absolute_error(y_true, y_pred)
    else:
        raise 'Task is not correct.'
        
def take_pred_from_task(pred, task):
    if task.name == 'binary' or task.name == 'reg':
        return pred[:, 0]
    elif task.name == 'multiclass' or task.name == 'multi:reg':
        return pred
    else:
        raise 'Task is not correct.'
        
def use_plr(USE_PLR):
    if USE_PLR:
        return "plr"
    else:
        return "cont"

### 0.2 Constants

Here we setup the constants to use in the kernel:
- `N_THREADS` - number of vCPUs for LightAutoML model creation
- `N_FOLDS` - number of folds in LightAutoML inner CV
- `RANDOM_STATE` - random seed for better reproducibility
- `TEST_SIZE` - houldout data part size 
- `TIMEOUT` - limit in seconds for model to train
- `TARGET_NAME` - target column name in dataset
- `TASK` - task name, 'reg', 'binary', 'multiclass', 'multi:reg'
- `ALGOS_FOR_BLEND` - algorithms used in blending
- `USE_PLR` - if True use PLR embedder for continuous features, else Basic Embedder
- `TRAIN_BS` - train batch size

In [4]:
RANDOM_STATE = 42
N_THREADS = 16

In [5]:
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

### 0.3 Data loading

In [6]:
train = pd.read_csv('../input/playground-series-s4e7/train.csv')
test = pd.read_csv('../input/playground-series-s4e7/test.csv')

In [7]:
#X_train, X_val = train_test_split(train, test_size=0.2, random_state=42, shuffle=True, stratify=train.Response)

In [12]:
task = Task('binary') #‘binary’ 
automl = TabularAutoML(
    task = task, 
    timeout = 600 * 3600,
    cpu_limit = 16,
    gpu_ids = '0',
    general_params = {"use_algos": [['denselight']]}, # ['nn', 'mlp', 'dense', 'denselight', 'resnet', 'snn', 'node', 'autoint', 'fttransformer'] or custom torch model
    nn_params = {
        "n_epochs": 10, 
        "bs": 1024, 
        "num_workers": 0, 
        "path_to_save": None, 
        "freeze_defaults": True,
        "cont_embedder": 'plr',
        'cat_embedder': 'weighted',
        'act_fun': 'SiLU',
        "hidden_size": [512, 128],
        'embedding_size': 32,
        'stop_by_metric': True,
        'verbose_bar': True,
        "snap_params": { 'k': 2, 'early_stopping': True, 'patience': 1, 'swa': True }
    },
    nn_pipeline_params = {"use_qnt": True, "use_te": False},
    reader_params = {'n_jobs': 16, 'cv': 5, 'random_state': 42, 'advanced_roles': True}
)

out_of_fold_predictions = automl.fit_predict(
    train, #valid_data=X_val,
    roles = {
        'target': 'Response',
        'drop': ['id']
    }, 
    verbose = 4
)

[23:04:47] Stdout logging level is DEBUG.
[23:04:47] Task: binary

[23:04:47] Start automl preset with listed constraints:
[23:04:47] - time: 2160000.00 seconds
[23:04:47] - CPU: 16 cores
[23:04:47] - memory: 16 GB

[23:04:47] Train data shape: (11504798, 12)

[23:05:19] Feats was rejected during automatic roles guess: []
[23:05:20] Layer 1 train process start. Time left 2159967.12 secs
[23:05:53] number of text features: 0 
[23:05:53] number of categorical features: 8 
[23:05:53] number of continuous features: 2 
[23:05:53] Start fitting Lvl_0_Pipe_0_Mod_0_TorchNN_denselight_0 ...
[23:05:53] Training params: {'num_workers': 0, 'pin_memory': False, 'max_length': 256, 'is_snap': False, 'input_bn': False, 'max_emb_size': 256, 'bert_name': None, 'pooling': 'cls', 'device': device(type='cuda', index=0), 'use_cont': True, 'use_cat': True, 'use_text': False, 'lang': 'en', 'deterministic': True, 'multigpu': False, 'random_state': 42, 'model': 'denselight', 'model_with_emb': False, 'path_to_sa

val: 100%|██████████| 2248/2248 [00:23<00:00, 97.57it/s]


[23:08:13] Epoch: 0, train loss: 0.24904334545135498, val loss: 0.2437402307987213, val metric: 0.8910446975933641


val: 100%|██████████| 2248/2248 [00:23<00:00, 94.76it/s]


[23:10:34] Epoch: 1, train loss: 0.2428310215473175, val loss: 0.2428431659936905, val metric: 0.8919118389207087


...

[00:01:35] Epoch: 3, train loss: 0.23930306732654572, val loss: 0.24263928830623627, val metric: 0.8923158550812593


val: 100%|██████████| 2248/2248 [00:23<00:00, 96.47it/s]


[00:03:56] Epoch: 4, train loss: 0.2378953993320465, val loss: 0.24316591024398804, val metric: 0.8920083818724441


val: 100%|██████████| 2248/2248 [00:26<00:00, 85.98it/s]


[00:04:23] Early stopping: val loss: 0.2423122525215149, val metric: 0.892639222234881
[00:04:28] Fitting Lvl_0_Pipe_0_Mod_0_TorchNN_denselight_0 finished. score = 0.8921928515951411
[00:04:28] Lvl_0_Pipe_0_Mod_0_TorchNN_denselight_0 fitting and predicting completed
[00:04:28] Time left 2156419.63 secs

[00:04:28] Layer 1 training completed.

[00:04:28] Automl preset training completed in 3580.46 seconds

[00:04:28] Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (5 averaged models Lvl_0_Pipe_0_Mod_0_TorchNN_denselight_0) 



In [13]:
roc_auc_score(train.Response, out_of_fold_predictions.data)

0.8921928515951411

In [14]:
pred = automl.predict(test)

test: 100%|██████████| 7491/7491 [00:49<00:00, 151.06it/s]


In [15]:
import joblib
joblib.dump((out_of_fold_predictions.data[:, 0], pred.data[:, 0]), 'denselight_5fold_oof_test_089219.jbl')
joblib.dump(automl, 'denselight_5fold_model_089219.jbl')

['denselight_5fold_model_089219.jbl']